In [12]:
def membership_functions(value,variable,descriptor): 
    if(variable=='dirt'):
        if(descriptor=='sd' and value>=0 and value<=50): 
            return (50-value)/50
        elif(descriptor=='md' and value>=0 and value<=50): 
            return value/50
        elif(descriptor=='md' and value>50 and value<=100): 
            return (100-value)/50
        elif(descriptor=='ld' and value>=50 and value<=100): 
            return (value-50)/50
    elif(variable=='grease'):
        if(descriptor=='ng' and value>=0 and value<=50): 
            return (50-value)/50
        elif(descriptor=='mg' and value>=0 and value<=50): 
            return value/50
        elif(descriptor=='mg' and value>50 and value<=100): 
            return (100-value)/50
        elif(descriptor=='lg' and value>=50 and value<=100):
            return (value-50)/50
def rule_base(dirt,grease): 
    if(dirt=='sd'):
        dirt = 0 
    elif(dirt=='md'):
        dirt = 1 
    elif(dirt=='ld'):
        dirt = 2 
    if(grease=='ng'):
        grease = 0 
    elif(grease=='mg'):
        grease = 1 
    elif(grease=='lg'):
        grease = 2

    rules = [['vs','m','l'],['s','m','l'],['m','l','vl']] 
    return rules[dirt][grease]
def mapping_dirt(percentage): 
    temp,temp1 = [],[]
    if(percentage>=0 and percentage<=50):
        # temp.append('sd') 
        temp.append(membership_functions(percentage,'dirt','sd')) 
        temp1.append('sd')
    if(percentage>=0 and percentage<=100): # temp.append('md')
        temp.append(membership_functions(percentage,'dirt','md')) 
        temp1.append('md')
    if(percentage>=50 and percentage<=100): # temp.append('ld')
        temp.append(membership_functions(percentage,'dirt','ld')) 
        temp1.append('ld')
    return temp,temp1

def mapping_grease(percentage): 
    temp,temp1 = [],[]
    if(percentage>=0 and percentage<=50): 
        temp.append(membership_functions(percentage,'grease','ng')) 
        temp1.append('ng')
    if(percentage>=0 and percentage<=100): 
        temp.append(membership_functions(percentage,'grease','mg')) 
        temp1.append('mg')
    if(percentage>=50 and percentage<=100): 
        temp.append(membership_functions(percentage,'grease','lg')) 
        temp1.append('lg')
    return temp,temp1

def mapping_washing_time(variable,value): 
    temp = []
    if(variable=='vs'):
        temp.append((-1)*((value*10)-10)) 
    elif(variable=='s'):
        temp.append(value*10) 
        temp.append((-1)*((value*15)-25))
    elif(variable=='m'): 
        temp.append((value*15)+10) 
        temp.append((-1)*((value*15)-40))
    elif(variable=='l'): 
        temp.append((value*15)+25) 
        temp.append((-1)*((value*20)-60))
    elif(variable=='vl'): 
        temp.append((value*20)+40)
    return sum(temp)/len(temp) 
import itertools as it
m,n = int(input("Enter dirt percentage : ")),int(input("Enter grease percentage : ")) 
l1,l1_rule = mapping_dirt(m)
l2,l2_rule = mapping_grease(n)
a = list(map(min,list(it.product(l1,l2)))) 
b = list(it.product(l1_rule,l2_rule))
temp1,temp2 = list(map(str,b[a.index(max(a))]))
print(mapping_washing_time(rule_base(temp1,temp2),max(a)))


Enter dirt percentage : 99
Enter grease percentage : 99
59.6


In [21]:
import numpy as np
from src.NeuralNetwork import NeuralNetwork
import src.utils as utils
def main():
    csv_filename = "data/seeds_dataset.csv"
    hidden_layers = [5] # number of nodes in hidden layers i.e. [layer1, layer2, ...]
    eta = 0.1 # learning rate
    n_epochs = 400 # number of training epochs
    n_folds = 4 # number of folds for cross-validation
    seed_crossval = 1 # seed for cross-validation
    seed_weights = 1 # seed for NN weight initialization 
    print("Reading '{}'...".format(csv_filename))
    X, y, n_classes = utils.read_csv(csv_filename, target_name="y", normalize=True)
    N, d = X.shape
    print(" -> X.shape = {}, y.shape = {}, n_classes = {}\n".format(X.shape, y.shape, n_classes))
    print("Neural network model:")
    print(" input_dim = {}".format(d))
    print(" hidden_layers = {}".format(hidden_layers))
    print(" output_dim = {}".format(n_classes))
    print(" eta = {}".format(eta))
    print(" n_epochs = {}".format(n_epochs))
    print(" n_folds = {}".format(n_folds))
    print(" seed_crossval = {}".format(seed_crossval))
    print(" seed_weights = {}\n".format(seed_weights)) 
    idx_all = np.arange(0, N)
    idx_folds = utils.crossval_folds(N, n_folds, seed=seed_crossval) 
    acc_train, acc_valid = list(), list() # training/test accuracy score
    print("Cross-validating with {} folds...".format(len(idx_folds)))
    for i, idx_valid in enumerate(idx_folds): 
        idx_train = np.delete(idx_all, idx_valid)
        X_train, y_train = X[idx_train], y[idx_train]
        X_valid, y_valid = X[idx_valid], y[idx_valid] 
        model = NeuralNetwork(input_dim=d, output_dim=n_classes,
        hidden_layers=hidden_layers, seed=seed_weights)
        model.train(X_train, y_train, eta=eta, n_epochs=n_epochs) 
        ypred_train = model.predict(X_train)
        ypred_valid = model.predict(X_valid) 
        acc_train.append(100*np.sum(y_train==ypred_train)/len(y_train))
        acc_valid.append(100*np.sum(y_valid==ypred_valid)/len(y_valid)) 
        print(" Fold {}/{}: acc_train = {:.2f}%, acc_valid = {:.2f}% (n_train = {}, n_valid = {})".format(i+1, n_folds, acc_train[-1], acc_valid[-1], len(X_train), len(X_valid))) 
        print(" -> acc_train_avg = {:.2f}%, acc_valid_avg = {:.2f}%".format(
        sum(acc_train)/float(len(acc_train)), sum(acc_valid)/float(len(acc_valid))))
if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'src'